In [8]:
from os.path import dirname
import os
import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb

In [180]:
pd.set_option('display.max_columns', 50)

In [141]:
%load_ext autoreload
%autoreload 2
from utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [138]:
MIMIC_DIR = os.path.join(os.getcwd(), "../../data/mimic/")
INPUT_DIR = os.path.join(os.getcwd(), "../../data/input/")

In [ ]:
sample_sub = pd.read_csv(INPUT_DIR + "sample_submission.csv")
sub_users = sample_sub[["KaiinID"]].drop_duplicates()

In [229]:
auction = pd.read_csv(INPUT_DIR + "auction.csv")
watch = pd.read_csv(INPUT_DIR + "watchlist.csv")
bid = pd.read_csv(INPUT_DIR + "shudounyuusatsu.csv")
bid_success = pd.read_csv(INPUT_DIR + "rakusatsu.csv")

In [230]:
access_date = datetime.datetime(2019, 10, 1, 0, 0, 0)
access_period = -7

In [231]:
auction = to_datetime(auction)
auction["CreateDate"] = auction["CreateDate"].apply(lambda d: (d - access_date).days)
watch = to_datetime(watch)
watch["TourokuDateDelta"] = watch["TourokuDate"].apply(lambda d: (d - access_date).days)
watch = watch.merge(auction, on="AuctionID")
bid = to_datetime(bid)
bid["ShudouNyuusatsuDateDelta"] = bid["ShudouNyuusatsuDate"].apply(lambda d: (d - access_date).days)
bid = bid.merge(auction, on="AuctionID")
bid_success = to_datetime(bid_success)
bid_success["RakusatsuDateDelta"] = bid_success["RakusatsuDate"].apply(lambda d: (d - access_date).days)
bid_success = bid_success.merge(auction, on="AuctionID")

In [370]:
# auction.to_pickle("auction.pkl")
# watch.to_pickle("watch.pkl")
# bid.to_pickle("bid.pkl")
# bid_success.to_pickle("bid_success.pkl")
auction= pd.read_pickle("auction.pkl")
watch= pd.read_pickle("watch.pkl")
bid= pd.read_pickle("bid.pkl")
bid_success= pd.read_pickle("bid_success.pkl")

In [390]:
[i for i in range(1,2)]

[1]

In [371]:
period_stack_num = 4

dataset_buf = []
for i in range(1, period_stack_num + 1):
    print(i)
    watch_actioned = (
        watch.query(f"(TourokuDateDelta >= {access_period} - 7 * {i}) & (TourokuDateDelta < {access_period} - 7 * {i - 1})")
    )
    bid_actioned = (
        bid
        .query(f"(ShudouNyuusatsuDateDelta >= {access_period} - 7 * {i}) & (ShudouNyuusatsuDateDelta < {access_period} - 7 * {i - 1})")
    )
    user_cand = (
        pd.concat([watch_actioned, bid_actioned], sort=False)[["KaiinID"]]
        .drop_duplicates()
    )
    watch_train = watch.query(f"TourokuDateDelta < {access_period} - 7 * {i}")
    bid_train = bid.query(f"ShudouNyuusatsuDateDelta < {access_period} - 7 * {i}")
    bid_success_train = bid_success.query(f"RakusatsuDateDelta < {access_period} - 7 * {i}")    

    train_dataset = build_dataset(user_cand, watch_train, bid_train, access_period - 7 * i)    
    
    watch_actioned["watch_actioned"] = 1
    bid_actioned["bid_actioned"] = 1
    train_dataset = (
        train_dataset
        .merge(watch_actioned[["KaiinID", "AuctionID", "watch_actioned"]], on=["KaiinID", "AuctionID"], how="left")
        .merge(bid_actioned[["KaiinID", "AuctionID", "bid_actioned"]], on=["KaiinID", "AuctionID"], how="left")
        .fillna(0)
    )
    
    dataset_buf.append(train_dataset)

train_datasets = pd.concat(dataset_buf)

1


/Users/nobuyuki.ishida/.pyenv/versions/3.7.5/lib/python3.7/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/nobuyuki.ishida/.pyenv/versions/3.7.5/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/nobuyuki.ishida/.pyenv/versions/3.7.5/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

2
3
4


In [372]:
def build_dataset(user_cand, watch, bid, period):
    
    # あるユーザーが現在までにアクションをした商品と同じ商品IDのオークションを抽出
    # これが関数内で作成する学習データの大元
    aucs_cand = (
        pd.concat([watch[["KaiinID", "ShouhinID"]], bid[["KaiinID", "ShouhinID"]]])
        .drop_duplicates()
        .merge(auction[["AuctionID", "ShouhinID"]], on="ShouhinID")[["KaiinID", "AuctionID"]]
        .drop_duplicates()
    )
    
    auction["TmpCreateDateDelta"] = auction["CreateDate"] - period    
    auc_cols = (
        ['AuctionID', 'ShouhinShubetsuID', 'ShouhinID', 'SaishuppinKaisuu',
       'ConditionID', 'BrandID', 'GenreID', 'GenreGroupID', 'LineID',
       'DanjobetsuID', 'TmpCreateDateDelta']
    )
    # 今回の対象ユーザーに絞る
    train_data = (
        user_cand
        .merge(aucs_cand, on="KaiinID")
        .merge(auction[auc_cols], on="AuctionID")
    )
    
    
    train_data = add_time_features(train_data, watch, "TourokuDateDelta", "watch", period)
    train_data = add_value_counts(
        train_data, watch, [["AuctionID"], ["ShouhinID"], ["BrandID"], ["LineID"], ["KaiinID", "ShouhinID"], 
        ["KaiinID", "BrandID"], ["KaiinID", "GenreGroupID"], ["KaiinID", "LineID"]], "watch"
    )

    train_data = add_time_features(train_data, bid, "ShudouNyuusatsuDateDelta", "bid", period)
    train_data = add_value_counts(
        train_data, bid, [["AuctionID"], ["ShouhinID"], ["BrandID"], ["LineID"], ["KaiinID", "ShouhinID"], 
        ["KaiinID", "BrandID"], ["KaiinID", "GenreGroupID"], ["KaiinID", "LineID"]], "bid"
    )    
    
    drop_cols = ["ShouhinShubetsuID", "ShouhinID", "BrandID", "GenreID", "GenreGroupID", "LineID"]
    
    train_data = train_data.drop(drop_cols, axis=1).fillna(-1)
    
    return train_data
    
def add_time_features(df, feature_df, time_col, prefix, period):
    tmp_time_col = f"Tmp_{time_col}"
    key_cols = ["KaiinID", "AuctionID"]
    feature_df[tmp_time_col] = feature_df[time_col] - period
    time_features = (
        feature_df
        .groupby(key_cols)[tmp_time_col]
        .agg(["count", "max", "min"])
        .rename(columns={"count": f"{prefix}_ua_cnt", "max": f"{prefix}_ua_newest", "min": f"{prefix}_ua_oldest"})
    )
    time_features[f"{prefix}_period"] = time_features[f"{prefix}_ua_newest"] - time_features[f"{prefix}_ua_oldest"]
    output = df.merge(time_features, on=key_cols, how="left")
    return output

def add_value_counts(df, feature_df, colsets, prefix):
    df_cp = df.copy()
    for colset in colsets:
        if len(colset) == 2:
            cnts = (
                feature_df[colset + ["AuctionID"]].groupby(colset, as_index=False).count()
                .rename(columns={"AuctionID": f"{prefix}_{colset[0]}_{colset[1]}_cnt"})
            )
        elif len(colset) == 1: 
            col = colset[0]
            cnts = feature_df[col].value_counts().reset_index().rename(columns={"index": col, col: f"{prefix}_{col}_cnt"})
        
        df_cp = df_cp.merge(cnts, on=colset, how="left")
    return df_cp


In [373]:
sub_mimic = build_dataset(sub_users, watch, bid, access_period)

In [388]:
train_datasets.shape

(13312315, 32)

In [375]:
train_datasets.query("(KaiinID == 134759) & (AuctionID == 1862947)")

,KaiinID,AuctionID,SaishuppinKaisuu,ConditionID,DanjobetsuID,TmpCreateDateDelta,watch_ua_cnt,watch_ua_newest,watch_ua_oldest,watch_period,watch_AuctionID_cnt,watch_ShouhinID_cnt,watch_BrandID_cnt,watch_LineID_cnt,watch_KaiinID_ShouhinID_cnt,watch_KaiinID_BrandID_cnt,watch_KaiinID_GenreGroupID_cnt,watch_KaiinID_LineID_cnt,bid_ua_cnt,bid_ua_newest,bid_ua_oldest,bid_period,bid_AuctionID_cnt,bid_ShouhinID_cnt,bid_BrandID_cnt,bid_LineID_cnt,bid_KaiinID_ShouhinID_cnt,bid_KaiinID_BrandID_cnt,bid_KaiinID_GenreGroupID_cnt,bid_KaiinID_LineID_cnt,watch_actioned,bid_actioned
0,134759,1862947,0,6,2,-389,1.0,-357.0,-357.0,0.0,9.0,9.0,9757.0,3874863,1.0,4.0,101.0,479.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,821.0,560636.0,-1.0,-1.0,17.0,65.0,0.0,0.0
1647206,134759,1862947,0,6,2,-382,1.0,-350.0,-350.0,0.0,9.0,9.0,9548.0,3794469,1.0,4.0,99.0,462.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,796.0,549648.0,-1.0,-1.0,16.0,63.0,0.0,0.0
147587,134759,1862947,0,6,2,-375,1.0,-343.0,-343.0,0.0,9.0,9.0,9397.0,3720858,1.0,4.0,96.0,452.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,785.0,539407.0,-1.0,-1.0,16.0,63.0,0.0,0.0
507082,134759,1862947,0,6,2,-368,1.0,-336.0,-336.0,0.0,9.0,9.0,9211.0,3650794,1.0,4.0,96.0,449.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,763.0,529031.0,-1.0,-1.0,16.0,63.0,0.0,0.0


In [378]:
sub_mimic.to_pickle("sub_mimic2.pkl")

In [379]:
train_datasets.to_pickle("train_datasets2.pkl")

In [380]:
train = pd.read_pickle("train_datasets2.pkl")
sub = pd.read_pickle("sub_mimic2.pkl")

In [381]:
lbg_params = {
    "objective": "binary",
    "nround": 500,
    "learning_rate": 0.01,
    "max_depth": 6,
    "num_leaves": 127
}

drop_cols = ["KaiinID", "AuctionID", "watch_actioned", "bid_actioned"]

watch_model = lgb.train(
    params=lbg_params,
    train_set=lgb.Dataset(np.array(train.drop(drop_cols, axis=1)), label = np.array(train["watch_actioned"]))
)
nyuusatsu_model = lgb.train(
    params=lbg_params,
    train_set=lgb.Dataset(np.array(train.drop(drop_cols, axis=1)), label = np.array(train["bid_actioned"]))    
)

In [382]:
watch_pred = watch_model.predict(np.array(sub.drop(["AuctionID", "KaiinID"], axis=1)))
nyuusatsu_pred = nyuusatsu_model.predict(np.array(sub.drop(["AuctionID", "KaiinID"], axis=1)))

In [383]:
sub["watch_pred"] = watch_pred
sub["nyuusatsu_pred"] = nyuusatsu_pred

In [384]:
sub_pred = sub[["KaiinID", "AuctionID", "watch_pred", "nyuusatsu_pred"]]
sub_pred["score"] = sub_pred["watch_pred"] * 0.2 + sub_pred["nyuusatsu_pred"] * 0.8
sub_ranks = sub_users.merge(sub_pred, on="KaiinID", how="left")[["KaiinID", "AuctionID", "score"]]

/Users/nobuyuki.ishida/.pyenv/versions/3.7.5/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [385]:
def comple_submit_auc(df):
    candidate_aucs = (
        df[["AuctionID", "score"]].groupby("AuctionID", as_index=False).mean().sort_values("score", ascending=False).iloc[:40,:]
    )
    candidate_aucs["score"] = 0
    target_users = df.groupby("KaiinID", as_index=False).count().query("score < 20")["KaiinID"].tolist()
    buf = []
    for user in target_users:
        candidate_aucs_tmp = candidate_aucs.copy()
        candidate_aucs_tmp["KaiinID"] = user
        buf.append(candidate_aucs_tmp)
    df_comple = pd.concat(buf)
    df_colmled = pd.concat([df, df_comple])
    return df_colmled

In [386]:
sub_ranks = comple_submit_auc(sub_ranks)

/Users/nobuyuki.ishida/.pyenv/versions/3.7.5/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]


In [387]:
sub_ranks.sort_values(['KaiinID', 'score'], ascending=[True, False], inplace=True)
sub_ranks['rank'] = sub_ranks.groupby('KaiinID')['score'].cumcount()
sub_valid = sub_ranks.query("rank < =19")
sub_valid.sort_values(['KaiinID', 'score'], ascending=[True, False], inplace=True)
sub_valid[["KaiinID", "AuctionID"]].to_csv("submit.csv", index=False)

/Users/nobuyuki.ishida/.pyenv/versions/3.7.5/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
